# ⚙️ Week 22, Day 3: Apache Airflow & ML Orchestration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gouthamgo/Learn-AI/blob/main/Phase-4-Advanced-AI/Week-22-Job-Critical-Skills/Day-3-Apache-Airflow-and-ML-Orchestration.ipynb)

## 🚀 Why This Matters

ML in production is **NOT** just training models in notebooks!

**Real production ML pipelines need**:
- 📊 **Data ingestion** from multiple sources
- 🔄 **Data transformation** and validation
- 🤖 **Model training** (potentially hours/days)
- ✅ **Model evaluation** and comparison
- 🚀 **Model deployment** if performance is good
- 🔁 **Retraining** on schedule (daily, weekly)
- 📧 **Alerting** when things fail

**Enter: ML Orchestration!**

### Job Market Reality

**Woolworths job listing**: ✅
> "Experience with Airflow or other orchestration tools for ML pipelines"

**Industry Stats**:
- 40% of ML Engineer jobs mention orchestration
- Airflow used by: Airbnb, Netflix, Spotify, Uber, Twitter
- Kubeflow used by: Google, Cisco, Bloomberg

**Real Impact**:
- Airbnb: 1000+ daily ML pipelines in Airflow
- Uber: Automated model retraining every 6 hours
- Netflix: Recommendation models retrain automatically

## 📋 What You'll Learn Today

1. **Apache Airflow Fundamentals** - DAGs, operators, scheduling
2. **ML Pipeline Orchestration** - End-to-end workflow
3. **Advanced Concepts** - Dynamic DAGs, XCom, branching
4. **Other Tools** - Kubeflow, Argo Workflows, Prefect
5. **Best Practices** - Error handling, monitoring, testing
6. **🏆 Project: Complete Automated ML Pipeline**

---

## Part 1: Apache Airflow Fundamentals

### What is Apache Airflow?

**Airflow** = Platform to programmatically author, schedule, and monitor workflows

**Key Concepts**:

1. **DAG (Directed Acyclic Graph)**
   - Defines workflow structure
   - Nodes = Tasks
   - Edges = Dependencies
   - Acyclic = No loops!

2. **Operators**
   - PythonOperator: Run Python functions
   - BashOperator: Run bash commands
   - EmailOperator: Send emails
   - Many more (Docker, Kubernetes, Spark, etc.)

3. **Tasks**
   - Instance of an operator
   - Can have dependencies: task_a >> task_b

4. **Schedule**
   - Cron expressions: `@daily`, `@hourly`, `0 0 * * *`

### Architecture

```
┌─────────────┐
│  Scheduler  │ ← Reads DAGs, schedules tasks
└─────┬───────┘
      │
      v
┌─────────────┐
│  Executor   │ ← Runs tasks (Local/Celery/Kubernetes)
└─────┬───────┘
      │
      v
┌─────────────┐
│   Workers   │ ← Execute task code
└─────────────┘
```

In [ ]:
# Install Airflow (Note: In production, use Docker or proper installation)
# For Colab, we'll demonstrate concepts without full installation

!pip install apache-airflow==2.7.0 -q
!pip install pandas numpy scikit-learn matplotlib seaborn -q

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries installed!")
print("\nNote: Full Airflow requires separate installation.")
print("We'll demonstrate DAG code that you can deploy in production.")

### Your First Airflow DAG

In [ ]:
# Example Airflow DAG (save as: dags/hello_world_dag.py)

hello_world_dag = '''
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime, timedelta

# Default arguments
default_args = {
    'owner': 'ml_engineer',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email': ['alerts@company.com'],
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
}

# Define Python tasks
def print_hello():
    print("Hello from Airflow!")
    return "Success!"

def print_date():
    print(f"Current date: {datetime.now()}")

# Create DAG
with DAG(
    dag_id='hello_world',
    default_args=default_args,
    description='Simple hello world DAG',
    schedule_interval='@daily',  # Run daily
    catchup=False,  # Don't backfill
) as dag:
    
    # Task 1: Print hello
    task_hello = PythonOperator(
        task_id='print_hello',
        python_callable=print_hello,
    )
    
    # Task 2: Print date
    task_date = PythonOperator(
        task_id='print_date',
        python_callable=print_date,
    )
    
    # Task 3: Bash command
    task_bash = BashOperator(
        task_id='run_bash',
        bash_command='echo "Running Bash task!"',
    )
    
    # Define dependencies (task flow)
    task_hello >> task_date >> task_bash  # Sequential execution
'''

print("📝 Example Airflow DAG:")
print("="*70)
print(hello_world_dag)
print("="*70)
print("\n💡 This DAG will:")
print("  1. Run daily (@daily schedule)")
print("  2. Execute 3 tasks sequentially")
print("  3. Retry failed tasks 2 times")
print("  4. Send email alerts on failure")

### Task Dependencies

**Multiple ways to define dependencies**:

```python
# Method 1: Bitshift operators
task_a >> task_b >> task_c  # Sequential: A → B → C

# Method 2: Parallel then merge
task_a >> [task_b, task_c] >> task_d
# A → B → D
#   → C → D

# Method 3: set_upstream/set_downstream
task_b.set_upstream(task_a)
task_c.set_downstream(task_d)

# Method 4: Complex dependencies
[task_a, task_b] >> task_c >> [task_d, task_e]
# A → C → D
# B → C → E
```

## Part 2: ML Pipeline with Airflow

### Complete ML Workflow

```
    ┌────────────────┐
    │  Extract Data  │
    └────────┬───────┘
             │
             v
    ┌────────────────┐
    │ Validate Data  │
    └────────┬───────┘
             │
             v
    ┌────────────────┐
    │ Transform Data │
    └────────┬───────┘
             │
    ┌────────┴────────┐
    │                 │
    v                 v
┌─────────┐    ┌─────────────┐
│ Train   │    │ Train       │
│ Model A │    │ Model B     │
└────┬────┘    └──────┬──────┘
     │                │
     └────────┬───────┘
              │
              v
    ┌────────────────┐
    │ Evaluate       │
    │ Best Model     │
    └────────┬───────┘
             │
             v
    ┌────────────────┐
    │ Deploy Model   │
    └────────┬───────┘
             │
             v
    ┌────────────────┐
    │ Send Alert     │
    └────────────────┘
```

In [ ]:
# Complete ML Pipeline DAG

ml_pipeline_dag = '''
from airflow import DAG
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.operators.email import EmailOperator
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import pickle
import json

# Configuration
DATA_PATH = '/data/customer_data.csv'
MODEL_PATH = '/models/'
METRICS_PATH = '/metrics/'
MIN_ACCURACY = 0.85

# Task 1: Extract data from database/API
def extract_data(**context):
    """
    Extract data from source (database, API, files)
    """
    print("Extracting data from source...")
    
    # Simulated data extraction
    # In production: pd.read_sql(), requests.get(), etc.
    df = pd.read_csv(DATA_PATH)
    
    print(f"Extracted {len(df)} rows")
    
    # Push data to XCom (Airflow's inter-task communication)
    context['ti'].xcom_push(key='row_count', value=len(df))
    
    # Save to temp location
    df.to_csv('/tmp/raw_data.csv', index=False)
    
    return 'Data extracted successfully'

# Task 2: Validate data quality
def validate_data(**context):
    """
    Check data quality: missing values, outliers, schema
    """
    print("Validating data quality...")
    
    df = pd.read_csv('/tmp/raw_data.csv')
    
    # Check for issues
    issues = []
    
    # Missing values
    missing_pct = df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100
    if missing_pct > 10:
        issues.append(f"High missing values: {missing_pct:.2f}%")
    
    # Duplicates
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        issues.append(f"Found {dup_count} duplicates")
    
    if issues:
        raise ValueError(f"Data quality issues: {issues}")
    
    print("✅ Data validation passed!")
    return 'Data is valid'

# Task 3: Transform data
def transform_data(**context):
    """
    Feature engineering, encoding, scaling
    """
    print("Transforming data...")
    
    df = pd.read_csv('/tmp/raw_data.csv')
    
    # Feature engineering (example)
    # df['new_feature'] = df['a'] * df['b']
    
    # Handle missing values
    df = df.fillna(df.mean())
    
    # Encoding categorical variables
    # df = pd.get_dummies(df, columns=['category'])
    
    # Save processed data
    df.to_csv('/tmp/processed_data.csv', index=False)
    
    print("✅ Data transformation complete!")
    return 'Data transformed'

# Task 4a: Train Random Forest
def train_random_forest(**context):
    """
    Train Random Forest model
    """
    print("Training Random Forest...")
    
    df = pd.read_csv('/tmp/processed_data.csv')
    X = df.drop('target', axis=1)
    y = df['target']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"Random Forest - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")
    
    # Save model and metrics
    with open('/tmp/rf_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    context['ti'].xcom_push(key='rf_accuracy', value=accuracy)
    context['ti'].xcom_push(key='rf_f1', value=f1)
    
    return accuracy

# Task 4b: Train Logistic Regression
def train_logistic_regression(**context):
    """
    Train Logistic Regression model
    """
    print("Training Logistic Regression...")
    
    df = pd.read_csv('/tmp/processed_data.csv')
    X = df.drop('target', axis=1)
    y = df['target']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train model
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"Logistic Regression - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")
    
    # Save model and metrics
    with open('/tmp/lr_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    context['ti'].xcom_push(key='lr_accuracy', value=accuracy)
    context['ti'].xcom_push(key='lr_f1', value=f1)
    
    return accuracy

# Task 5: Select best model
def select_best_model(**context):
    """
    Compare models and select best one
    """
    ti = context['ti']
    
    rf_acc = ti.xcom_pull(key='rf_accuracy', task_ids='train_random_forest')
    lr_acc = ti.xcom_pull(key='lr_accuracy', task_ids='train_logistic_regression')
    
    print(f"Random Forest Accuracy: {rf_acc:.4f}")
    print(f"Logistic Regression Accuracy: {lr_acc:.4f}")
    
    if rf_acc > lr_acc:
        best_model = 'random_forest'
        best_acc = rf_acc
    else:
        best_model = 'logistic_regression'
        best_acc = lr_acc
    
    print(f"\n🏆 Best Model: {best_model} (Accuracy: {best_acc:.4f})")
    
    # Check if meets threshold
    ti.xcom_push(key='best_model', value=best_model)
    ti.xcom_push(key='best_accuracy', value=best_acc)
    
    if best_acc >= MIN_ACCURACY:
        return 'deploy_model'  # Branch to deployment
    else:
        return 'send_failure_alert'  # Branch to alert

# Task 6a: Deploy model
def deploy_model(**context):
    """
    Deploy best model to production
    """
    ti = context['ti']
    best_model = ti.xcom_pull(key='best_model', task_ids='select_best_model')
    
    print(f"Deploying {best_model} to production...")
    
    # Copy model to production path
    # shutil.copy(f'/tmp/{best_model}.pkl', f'{MODEL_PATH}/production_model.pkl')
    
    # Update model registry
    # Update API endpoint
    # Restart service
    
    print("✅ Model deployed successfully!")
    return 'Deployment successful'

# Task 6b: Send failure alert
def send_failure_alert(**context):
    """
    Alert when model doesn't meet threshold
    """
    ti = context['ti']
    best_acc = ti.xcom_pull(key='best_accuracy', task_ids='select_best_model')
    
    message = f"""
    ⚠️ ML Pipeline Alert
    
    Model accuracy ({best_acc:.4f}) is below threshold ({MIN_ACCURACY}).
    Manual review required.
    
    Pipeline run: {context['execution_date']}
    """
    
    print(message)
    # Send email/Slack notification
    
    return 'Alert sent'

# Define DAG
default_args = {
    'owner': 'ml_team',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email': ['ml-team@company.com'],
    'email_on_failure': True,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
}

with DAG(
    dag_id='ml_training_pipeline',
    default_args=default_args,
    description='Automated ML training and deployment',
    schedule_interval='0 2 * * *',  # Daily at 2 AM
    catchup=False,
) as dag:
    
    # Define tasks
    extract = PythonOperator(
        task_id='extract_data',
        python_callable=extract_data,
    )
    
    validate = PythonOperator(
        task_id='validate_data',
        python_callable=validate_data,
    )
    
    transform = PythonOperator(
        task_id='transform_data',
        python_callable=transform_data,
    )
    
    train_rf = PythonOperator(
        task_id='train_random_forest',
        python_callable=train_random_forest,
    )
    
    train_lr = PythonOperator(
        task_id='train_logistic_regression',
        python_callable=train_logistic_regression,
    )
    
    select_model = BranchPythonOperator(
        task_id='select_best_model',
        python_callable=select_best_model,
    )
    
    deploy = PythonOperator(
        task_id='deploy_model',
        python_callable=deploy_model,
    )
    
    alert = PythonOperator(
        task_id='send_failure_alert',
        python_callable=send_failure_alert,
    )
    
    # Define pipeline flow
    extract >> validate >> transform >> [train_rf, train_lr] >> select_model
    select_model >> deploy
    select_model >> alert
'''

print("📝 Complete ML Pipeline DAG:")
print("="*70)
print("This is a production-ready ML pipeline that:")
print("")
print("1. ✅ Extracts data from source")
print("2. ✅ Validates data quality")
print("3. ✅ Transforms/engineers features")
print("4. ✅ Trains multiple models in parallel")
print("5. ✅ Selects best model")
print("6. ✅ Deploys if accuracy threshold met")
print("7. ✅ Sends alerts if threshold not met")
print("8. ✅ Runs daily at 2 AM")
print("9. ✅ Retries failed tasks 3 times")
print("10. ✅ Sends email on failures")
print("="*70)

## Part 3: Advanced Airflow Concepts

### XCom (Cross-Communication)

**Share data between tasks**:

```python
# Push data to XCom
def task_a(**context):
    context['ti'].xcom_push(key='my_key', value='my_value')

# Pull data from XCom
def task_b(**context):
    value = context['ti'].xcom_pull(key='my_key', task_ids='task_a')
    print(f"Received: {value}")
```

**Note**: XCom limited to small data (<1MB). For large data, use files/databases.

### Dynamic DAGs

**Generate tasks dynamically**:

```python
# Train models for multiple products
products = ['electronics', 'clothing', 'food']

for product in products:
    task = PythonOperator(
        task_id=f'train_{product}_model',
        python_callable=train_model,
        op_kwargs={'product': product},
    )
```

### Sensors

**Wait for conditions**:

```python
from airflow.sensors.filesystem import FileSensor

wait_for_file = FileSensor(
    task_id='wait_for_data',
    filepath='/data/new_data.csv',
    poke_interval=60,  # Check every 60 seconds
    timeout=3600,  # Timeout after 1 hour
)
```

### TaskGroups

**Organize related tasks**:

```python
from airflow.utils.task_group import TaskGroup

with TaskGroup('data_prep') as data_prep:
    extract = PythonOperator(...)
    validate = PythonOperator(...)
    transform = PythonOperator(...)
    extract >> validate >> transform

with TaskGroup('model_training') as model_training:
    train_rf = PythonOperator(...)
    train_lr = PythonOperator(...)

data_prep >> model_training
```

## Part 4: Other Orchestration Tools

### Kubeflow Pipelines

**Kubernetes-native ML orchestration**

**Advantages**:
- ✅ Designed specifically for ML
- ✅ Native Kubernetes integration
- ✅ Experiment tracking built-in
- ✅ Used by Google, Cisco, Bloomberg

**Example**:

In [ ]:
kubeflow_example = '''
import kfp
from kfp import dsl
from kfp.components import create_component_from_func

# Define pipeline components
@create_component_from_func
def load_data_op() -> str:
    """Load data component"""
    import pandas as pd
    df = pd.read_csv('/data/dataset.csv')
    return '/tmp/data.csv'

@create_component_from_func
def train_model_op(data_path: str) -> str:
    """Train model component"""
    from sklearn.ensemble import RandomForestClassifier
    import pickle
    
    # Load data, train model
    model = RandomForestClassifier()
    # model.fit(X, y)
    
    # Save model
    with open('/tmp/model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    return '/tmp/model.pkl'

@create_component_from_func
def deploy_model_op(model_path: str):
    """Deploy model component"""
    print(f"Deploying model from {model_path}")
    # Deployment logic

# Define pipeline
@dsl.pipeline(
    name='ML Training Pipeline',
    description='Train and deploy ML model'
)
def ml_pipeline():
    # Create pipeline steps
    load_data_task = load_data_op()
    train_model_task = train_model_op(load_data_task.output)
    deploy_model_task = deploy_model_op(train_model_task.output)

# Compile pipeline
kfp.compiler.Compiler().compile(
    pipeline_func=ml_pipeline,
    package_path='ml_pipeline.yaml'
)

# Deploy to Kubeflow
client = kfp.Client()
client.create_run_from_pipeline_func(
    ml_pipeline,
    arguments={},
    experiment_name='ml_experiment'
)
'''

print("📝 Kubeflow Pipeline Example:")
print("="*70)
print(kubeflow_example)
print("="*70)

### Prefect

**Modern workflow orchestration**

**Advantages**:
- ✅ Python-first (more Pythonic than Airflow)
- ✅ Better error handling
- ✅ Dynamic workflows
- ✅ Cloud-hosted option

**Example**:

In [ ]:
prefect_example = '''
from prefect import task, Flow
from prefect.schedules import CronSchedule
import pandas as pd

@task
def extract_data():
    df = pd.read_csv('/data/dataset.csv')
    return df

@task
def transform_data(df):
    # Feature engineering
    df_transformed = df.fillna(0)
    return df_transformed

@task
def train_model(df):
    from sklearn.ensemble import RandomForestClassifier
    X = df.drop('target', axis=1)
    y = df['target']
    
    model = RandomForestClassifier()
    model.fit(X, y)
    return model

@task
def deploy_model(model):
    # Deployment logic
    print("Model deployed!")

# Define flow
schedule = CronSchedule("0 2 * * *")  # Daily at 2 AM

with Flow("ml_pipeline", schedule=schedule) as flow:
    data = extract_data()
    transformed = transform_data(data)
    model = train_model(transformed)
    deploy_model(model)

# Register flow
flow.register(project_name="ml_project")

# Run flow
flow.run()
'''

print("📝 Prefect Pipeline Example:")
print("="*70)
print(prefect_example)
print("="*70)
print("\n💡 Prefect advantages:")
print("  - More Pythonic syntax")
print("  - Better error handling")
print("  - Native Python dataflow")
print("  - Easier to test locally")

### Tool Comparison

| Feature | Airflow | Kubeflow | Prefect | Argo Workflows |
|---------|---------|----------|---------|----------------|
| **Best For** | General workflows | ML on Kubernetes | Python workflows | Kubernetes workflows |
| **Learning Curve** | Medium | High | Low | Medium |
| **ML-Specific** | ❌ | ✅ | ❌ | ❌ |
| **Kubernetes** | Optional | Native | Optional | Native |
| **UI** | ✅ Good | ✅ Good | ✅ Excellent | ✅ Good |
| **Community** | ✅ Large | Medium | Growing | Medium |
| **Used By** | Airbnb, Netflix | Google, Cisco | Prefect Cloud | Intuit, Adobe |

**When to use each**:
- **Airflow**: General data engineering + ML (most common)
- **Kubeflow**: ML-focused, already on Kubernetes
- **Prefect**: New projects, Python-heavy teams
- **Argo**: Kubernetes-native, CI/CD + ML

## Part 5: Best Practices

### 1. Idempotency

**Tasks should produce same result when run multiple times**

```python
# ❌ BAD: Appends every time
def bad_task():
    df = pd.read_csv('data.csv')
    df.to_csv('output.csv', mode='a')  # Appends!

# ✅ GOOD: Overwrites
def good_task():
    df = pd.read_csv('data.csv')
    df.to_csv('output.csv', mode='w')  # Overwrites
```

### 2. Error Handling

```python
@task(retries=3, retry_delay=timedelta(minutes=5))
def robust_task():
    try:
        # Task logic
        result = risky_operation()
        return result
    except SpecificError as e:
        logger.error(f"Task failed: {e}")
        raise  # Re-raise for Airflow to retry
```

### 3. Monitoring & Alerting

```python
from airflow.operators.email import EmailOperator

send_alert = EmailOperator(
    task_id='send_alert',
    to='ml-team@company.com',
    subject='Pipeline Failed',
    html_content='Pipeline {{ dag.dag_id }} failed.',
    trigger_rule='one_failed',  # Trigger if any upstream task fails
)
```

### 4. Testing DAGs

```python
# Test DAG structure
def test_dag_loaded():
    from airflow.models import DagBag
    dagbag = DagBag()
    assert 'ml_pipeline' in dagbag.dags
    assert len(dagbag.import_errors) == 0

# Test task logic
def test_transform_data():
    df = pd.DataFrame({'a': [1, 2, 3]})
    result = transform_data(df)
    assert result is not None
    assert len(result) == 3
```

### 5. Secrets Management

```python
from airflow.hooks.base import BaseHook

# ❌ BAD: Hardcoded credentials
db_conn = psycopg2.connect(
    host='db.example.com',
    password='mysecret123'
)

# ✅ GOOD: Use Airflow connections
connection = BaseHook.get_connection('my_postgres')
db_conn = psycopg2.connect(
    host=connection.host,
    password=connection.password
)
```

### 6. Resource Management

```python
# Set resource limits
heavy_task = PythonOperator(
    task_id='train_large_model',
    python_callable=train_model,
    pool='gpu_pool',  # Use dedicated resource pool
    queue='gpu_queue',  # Use specific queue
)
```

## 🏆 CAPSTONE PROJECT: Production ML Pipeline

### Project Goal
Build a complete production ML pipeline with:
- Data ingestion from multiple sources
- Data validation and quality checks
- Feature engineering
- Multiple model training
- Model evaluation and selection
- Automated deployment
- Monitoring and alerting
- Scheduled retraining

### System Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    AIRFLOW SCHEDULER                         │
└─────────────────┬───────────────────────────────────────────┘
                  │
    ┌─────────────┼─────────────┐
    │             │             │
    v             v             v
┌─────────┐  ┌─────────┐  ┌─────────┐
│ Worker 1│  │ Worker 2│  │ Worker 3│
└─────────┘  └─────────┘  └─────────┘
    │             │             │
    v             v             v
┌──────────────────────────────────────┐
│        DATA SOURCES                   │
│  • Database  • API  • S3  • CSV      │
└──────────────────────────────────────┘
             │
             v
┌──────────────────────────────────────┐
│      FEATURE STORE (Redis)           │
└──────────────────────────────────────┘
             │
             v
┌──────────────────────────────────────┐
│      MODEL REGISTRY (MLflow)         │
└──────────────────────────────────────┘
             │
             v
┌──────────────────────────────────────┐
│      DEPLOYMENT (Kubernetes)         │
└──────────────────────────────────────┘
```

In [ ]:
# Complete Production ML Pipeline

production_pipeline = '''
"""
Production ML Pipeline - Customer Churn Prediction

Schedule: Daily at 2 AM
Purpose: Retrain churn model with latest data
Deployment: Automatic if accuracy > 85%
"""

from airflow import DAG
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.providers.amazon.aws.hooks.s3 import S3Hook
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn
import pickle
import json
from typing import Dict, Any
import logging

# Configuration
CONFIG = {
    'db_conn_id': 'postgres_production',
    's3_bucket': 'ml-models-prod',
    'model_registry': 'mlflow_server',
    'min_accuracy': 0.85,
    'min_rows': 1000,
    'alert_email': 'ml-team@company.com',
}

logger = logging.getLogger(__name__)

# Task 1: Extract data from multiple sources
def extract_data(**context):
    """
    Extract customer data from:
    - PostgreSQL (customer info)
    - S3 (behavioral data)
    - API (realtime features)
    """
    logger.info("Starting data extraction...")
    
    # Extract from PostgreSQL
    pg_hook = PostgresHook(postgres_conn_id=CONFIG['db_conn_id'])
    sql = """
        SELECT customer_id, age, tenure, monthly_charges, total_charges,
               contract_type, payment_method, churn
        FROM customers
        WHERE last_updated >= CURRENT_DATE - INTERVAL '7 days'
    """
    df_customers = pg_hook.get_pandas_df(sql)
    
    # Extract from S3 (behavioral data)
    s3_hook = S3Hook(aws_conn_id='aws_default')
    obj = s3_hook.get_key(
        key='behavioral_features/latest.csv',
        bucket_name=CONFIG['s3_bucket']
    )
    df_behavior = pd.read_csv(obj.get()['Body'])
    
    # Merge datasets
    df = df_customers.merge(df_behavior, on='customer_id', how='left')
    
    # Save to temp
    df.to_csv('/tmp/raw_data.csv', index=False)
    
    # Push metadata
    context['ti'].xcom_push(key='row_count', value=len(df))
    context['ti'].xcom_push(key='extraction_time', value=datetime.now().isoformat())
    
    logger.info(f"Extracted {len(df)} rows")
    return 'Data extracted'

# Task 2: Data validation
def validate_data(**context):
    """
    Comprehensive data quality checks
    """
    logger.info("Starting data validation...")
    
    df = pd.read_csv('/tmp/raw_data.csv')
    
    validation_results = {
        'checks_passed': [],
        'checks_failed': [],
        'warnings': []
    }
    
    # Check 1: Minimum row count
    if len(df) < CONFIG['min_rows']:
        validation_results['checks_failed'].append(
            f"Insufficient data: {len(df)} < {CONFIG['min_rows']}"
        )
    else:
        validation_results['checks_passed'].append('Row count check')
    
    # Check 2: Missing values
    missing_pct = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    if missing_pct > 20:
        validation_results['checks_failed'].append(
            f"High missing values: {missing_pct:.2f}%"
        )
    elif missing_pct > 10:
        validation_results['warnings'].append(
            f"Moderate missing values: {missing_pct:.2f}%"
        )
    else:
        validation_results['checks_passed'].append('Missing values check')
    
    # Check 3: Duplicates
    dup_count = df.duplicated(subset=['customer_id']).sum()
    if dup_count > 0:
        validation_results['warnings'].append(f"Found {dup_count} duplicates")
    else:
        validation_results['checks_passed'].append('Duplicate check')
    
    # Check 4: Data types
    if df['age'].dtype not in [np.int64, np.float64]:
        validation_results['checks_failed'].append("Invalid age data type")
    else:
        validation_results['checks_passed'].append('Data type check')
    
    # Check 5: Target distribution
    churn_rate = df['churn'].mean()
    if churn_rate < 0.05 or churn_rate > 0.5:
        validation_results['warnings'].append(
            f"Unusual churn rate: {churn_rate:.2%}"
        )
    else:
        validation_results['checks_passed'].append('Target distribution check')
    
    # Save validation report
    context['ti'].xcom_push(key='validation_results', value=validation_results)
    
    # Fail if critical checks failed
    if validation_results['checks_failed']:
        raise ValueError(f"Data validation failed: {validation_results['checks_failed']}")
    
    logger.info(f"✅ Validation passed! {len(validation_results['checks_passed'])} checks")
    
    if validation_results['warnings']:
        logger.warning(f"Warnings: {validation_results['warnings']}")
    
    return 'Data validated'

# Task 3: Feature engineering
def engineer_features(**context):
    """
    Advanced feature engineering
    """
    logger.info("Starting feature engineering...")
    
    df = pd.read_csv('/tmp/raw_data.csv')
    
    # Handle missing values
    df = df.fillna(df.median(numeric_only=True))
    
    # Create new features
    df['avg_monthly_charges'] = df['total_charges'] / (df['tenure'] + 1)
    df['tenure_years'] = df['tenure'] / 12
    df['is_new_customer'] = (df['tenure'] < 12).astype(int)
    df['is_high_value'] = (df['monthly_charges'] > df['monthly_charges'].quantile(0.75)).astype(int)
    
    # Encode categorical variables
    df = pd.get_dummies(df, columns=['contract_type', 'payment_method'], drop_first=True)
    
    # Save processed data
    df.to_csv('/tmp/processed_data.csv', index=False)
    
    # Push feature metadata
    context['ti'].xcom_push(key='feature_count', value=len(df.columns) - 1)
    
    logger.info(f"✅ Feature engineering complete! {len(df.columns) - 1} features")
    return 'Features engineered'

# Task 4a: Train Random Forest
def train_random_forest(**context):
    logger.info("Training Random Forest...")
    
    df = pd.read_csv('/tmp/processed_data.csv')
    X = df.drop(['customer_id', 'churn'], axis=1)
    y = df['churn']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Train with MLflow tracking
    with mlflow.start_run(run_name='random_forest'):
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=20,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba)
        }
        
        # Log to MLflow
        mlflow.log_params(model.get_params())
        mlflow.log_metrics(metrics)
        mlflow.sklearn.log_model(model, "model")
    
    # Save model
    with open('/tmp/rf_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    # Push metrics
    for key, value in metrics.items():
        context['ti'].xcom_push(key=f'rf_{key}', value=value)
    
    logger.info(f"✅ RF trained! Accuracy: {metrics['accuracy']:.4f}")
    return metrics['accuracy']

# Similar tasks for other models (GradientBoosting, LogisticRegression)
# ...

# Task 5: Select best model
def select_best_model(**context):
    ti = context['ti']
    
    # Pull metrics from all models
    models = ['rf', 'gb', 'lr']
    results = {}
    
    for model in models:
        results[model] = {
            'accuracy': ti.xcom_pull(key=f'{model}_accuracy', task_ids=f'train_{model}'),
            'f1': ti.xcom_pull(key=f'{model}_f1', task_ids=f'train_{model}'),
            'roc_auc': ti.xcom_pull(key=f'{model}_roc_auc', task_ids=f'train_{model}')
        }
    
    # Select based on F1 score (balanced metric)
    best_model = max(results, key=lambda x: results[x]['f1'])
    best_metrics = results[best_model]
    
    logger.info(f"🏆 Best model: {best_model}")
    logger.info(f"Metrics: {best_metrics}")
    
    # Push results
    ti.xcom_push(key='best_model', value=best_model)
    ti.xcom_push(key='best_metrics', value=best_metrics)
    
    # Decision: deploy or alert
    if best_metrics['accuracy'] >= CONFIG['min_accuracy']:
        return 'deploy_model'
    else:
        return 'send_alert'

# Task 6a: Deploy model
def deploy_model(**context):
    ti = context['ti']
    best_model = ti.xcom_pull(key='best_model', task_ids='select_best_model')
    
    logger.info(f"Deploying {best_model} to production...")
    
    # 1. Upload to S3
    s3_hook = S3Hook(aws_conn_id='aws_default')
    s3_hook.load_file(
        filename=f'/tmp/{best_model}_model.pkl',
        key=f'models/churn_model_latest.pkl',
        bucket_name=CONFIG['s3_bucket'],
        replace=True
    )
    
    # 2. Update model registry
    mlflow.register_model(
        f"runs:/{context['run_id']}/model",
        "churn_prediction_model"
    )
    
    # 3. Update Kubernetes deployment
    # kubectl set image deployment/ml-api ml-api=ml-api:${NEW_VERSION}
    
    logger.info("✅ Model deployed successfully!")
    return 'Deployment complete'

# Task 6b: Send alert
def send_alert(**context):
    ti = context['ti']
    best_metrics = ti.xcom_pull(key='best_metrics', task_ids='select_best_model')
    
    message = f"""
    ⚠️ ML Pipeline Alert - Manual Review Required
    
    Model accuracy ({best_metrics['accuracy']:.4f}) is below threshold ({CONFIG['min_accuracy']}).
    
    Best Model Metrics:
    - Accuracy: {best_metrics['accuracy']:.4f}
    - F1 Score: {best_metrics['f1']:.4f}
    - ROC AUC: {best_metrics['roc_auc']:.4f}
    
    Pipeline Run: {context['execution_date']}
    DAG: {context['dag'].dag_id}
    
    Action Required: Review model performance and data quality.
    """
    
    logger.warning(message)
    # Send to Slack/Email/PagerDuty
    
    return 'Alert sent'

# Define DAG
default_args = {
    'owner': 'ml_platform_team',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email': [CONFIG['alert_email']],
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
    'retry_exponential_backoff': True,
    'max_retry_delay': timedelta(minutes=30),
}

with DAG(
    dag_id='production_churn_prediction_pipeline',
    default_args=default_args,
    description='Production ML pipeline for customer churn prediction',
    schedule_interval='0 2 * * *',  # Daily at 2 AM
    catchup=False,
    tags=['ml', 'production', 'churn'],
) as dag:
    
    extract = PythonOperator(task_id='extract_data', python_callable=extract_data)
    validate = PythonOperator(task_id='validate_data', python_callable=validate_data)
    engineer = PythonOperator(task_id='engineer_features', python_callable=engineer_features)
    
    train_rf = PythonOperator(task_id='train_rf', python_callable=train_random_forest)
    train_gb = PythonOperator(task_id='train_gb', python_callable=train_gradient_boosting)
    train_lr = PythonOperator(task_id='train_lr', python_callable=train_logistic_regression)
    
    select = BranchPythonOperator(task_id='select_best_model', python_callable=select_best_model)
    
    deploy = PythonOperator(task_id='deploy_model', python_callable=deploy_model)
    alert = PythonOperator(task_id='send_alert', python_callable=send_alert)
    
    # Pipeline flow
    extract >> validate >> engineer >> [train_rf, train_gb, train_lr] >> select
    select >> deploy
    select >> alert
'''

print("📝 Complete Production ML Pipeline:")
print("="*70)
print("\n✅ This is a PRODUCTION-READY pipeline with:")
print("")
print("1. Multi-source data extraction (PostgreSQL + S3 + API)")
print("2. Comprehensive data validation (5+ checks)")
print("3. Advanced feature engineering")
print("4. Parallel model training (3 models)")
print("5. MLflow experiment tracking")
print("6. Automated model selection")
print("7. Conditional deployment (accuracy threshold)")
print("8. S3 + MLflow + Kubernetes deployment")
print("9. Comprehensive error handling & retries")
print("10. Monitoring & alerting")
print("11. Scheduled retraining (daily)")
print("")
print("This is what ML Engineers build in real companies! 🚀")
print("="*70)

## 🎓 Key Takeaways

### What You've Learned

1. **Airflow Fundamentals**
   - DAGs, operators, tasks, dependencies
   - Scheduling with cron expressions
   - XCom for inter-task communication

2. **ML Pipeline Design**
   - Extract → Validate → Transform → Train → Deploy
   - Parallel model training
   - Conditional branching

3. **Advanced Concepts**
   - Dynamic DAGs
   - Sensors for waiting
   - TaskGroups for organization
   - BranchOperator for conditional logic

4. **Other Tools**
   - Kubeflow for ML on Kubernetes
   - Prefect for Python-first workflows
   - Argo for cloud-native workflows

5. **Best Practices**
   - Idempotency
   - Error handling & retries
   - Monitoring & alerting
   - Testing DAGs
   - Secrets management

6. **Production Skills**
   - Multi-source data integration
   - Data quality validation
   - MLflow experiment tracking
   - Automated deployment
   - Scheduled retraining

### Interview-Ready Skills ✅

**You can now answer:**
- "Explain how you would orchestrate an ML pipeline"
- "What is Airflow and when would you use it?"
- "How do you handle failures in ML pipelines?"
- "Describe a production ML pipeline you've built"
- "What is the difference between Airflow and Kubeflow?"

**You can now build:**
- Automated ML training pipelines
- Multi-model evaluation systems
- Scheduled model retraining
- Production deployment workflows

### Real-World Applications

**Tech Companies**:
- Airbnb: 1000+ Airflow DAGs for ML
- Uber: Automated model retraining
- Netflix: Recommendation model pipelines
- Spotify: Music recommendation orchestration

**Financial Services**:
- Fraud detection model updates
- Credit scoring automation
- Risk model retraining

**E-commerce** (Woolworths, Amazon):
- Demand forecasting pipelines
- Price optimization automation
- Recommendation system updates

---

## 🚀 Next Steps

1. **Set up Airflow locally**: Use Docker Compose
2. **Build your own pipeline**: Automate a personal ML project
3. **Learn Kubernetes**: For scalable orchestration
4. **Explore Kubeflow**: For ML-specific workflows
5. **Add to portfolio**: Show end-to-end automation

---

## 📚 Additional Resources

**Documentation**:
- [Apache Airflow Docs](https://airflow.apache.org/docs/)
- [Kubeflow Docs](https://www.kubeflow.org/docs/)
- [Prefect Docs](https://docs.prefect.io/)

**Courses**:
- "Apache Airflow: The Hands-On Guide" (Udemy)
- "Building Production ML Pipelines" (Google Cloud)

**Books**:
- "Data Pipelines with Apache Airflow"
- "ML Engineering" by Andriy Burkov

---

## ✅ Job Market Relevance

**Woolworths ML Engineer**: ✅
> "Experience with Airflow or other orchestration tools for ML pipelines"

**Industry Stats**: ✅
- 40% of ML Engineer jobs mention orchestration
- Airflow is #1 orchestration tool
- Production ML = Automation!

**You are now job-ready for ML orchestration roles!** 🎉

---

## 🎊 Congratulations!

**You've completed Week 22: Job-Critical Skills!**

You now have:
- ✅ Recommender Systems (Day 1)
- ✅ Time Series Forecasting (Day 2)
- ✅ ML Orchestration (Day 3)

**These are the TOP 3 missing skills from job postings.**

**Combined with Week 21, you now have 100% coverage of 2025 job market requirements!**

---

**You are ready to interview for Senior ML Engineer roles!** 🚀

Go build amazing things! 💪